In [1]:
!pip install asterix_decoder

  Using cached asterix_decoder-0.7.3-cp39-cp39-linux_x86_64.whl


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import asterix
import dpkt
%matplotlib inline

In [3]:
# Read example file from packet resources
sample_filename = ['09-10.pcap','09-17.pcap', '09-24.pcap','10-01.pcap','10-08.pcap','10-15.pcap','10-22.pcap','10-29.pcap','11-05.pcap','11-12.pcap']
asterix_data = []
TS=[]

In [4]:
for file in sample_filename:
    fichier = file
    f = open(fichier, "rb")
    pcap = dpkt.pcap.Reader(f)
    for ts, buf in pcap:
        eth = dpkt.ethernet.Ethernet(buf)
        data = eth.ip.udp.data
        asterix_data.append((data,file[:-5],ts))


In [5]:
Dataset = [] 
Dict = {}
print(len(asterix_data))
print(type(asterix_data))
print(asterix_data[0])

50705
<class 'list'>
(b'0\x006\xff\xff\x02\x08\n\x12\xdd\xaf\xa0\xaa\xbf\x0cy\x02\x00\x05\xa0`\x07\xc3@p\xa7\x084\xf2\xe1\x08 \x01\x10\x03\n\x80\xf5\x00\x00\x10\x07W\x19\xb8Qh\x08%\xb3IF"\xf5"\x00\x12\xf6\x08\n\x02\x12\xdd\xb6\x10\x84\x00 \x00\x84\x00\x00', '09-10', 1599705660.049571)


In [6]:
x = asterix.parse(asterix_data[0][0])
x

[{'category': 48,
  'len': 51,
  'crc': '63365EE5',
  'ts': 0,
  'hexdata': 'FFFF02080A12DDAFA0AABF0C79020005A06007C34070A70834F2E108200110030A80F5000010075719B851680825B3494622F5',
  'I010': {'SAC': {'desc': 'System Area Code', 'val': 8},
   'SIC': {'desc': 'System Identification Code', 'val': 10}},
  'I140': {'ToD': {'desc': 'Time Of Day', 'val': 9659.3671875}},
  'I020': {'TYP': {'desc': 'TYP',
    'val': 5,
    'meaning': 'Single ModeS Roll-Call'},
   'SIM': {'desc': 'SIM', 'val': 0, 'meaning': 'Actual target report'},
   'RDP': {'desc': 'RDP', 'val': 0, 'meaning': 'Report from RDP Chain 1'},
   'SPI': {'desc': 'SPI', 'val': 0, 'meaning': 'Absence of SPI'},
   'RAB': {'desc': 'RAB',
    'val': 0,
    'meaning': 'Report from aircraft transponder'},
   'FX': {'desc': 'FX', 'val': 0, 'meaning': 'End of Data Item'}},
  'I040': {'RHO': {'desc': '', 'val': 170.74609375, 'max': 256.0},
   'THETA': {'desc': '', 'val': 17.5396728515625}},
  'I070': {'V': {'desc': '', 'val': 0, 'meaning': 'C

In [7]:
for k in asterix_data:
    pars = asterix.parse(k[0])
    for i in range(len(pars)):
        parsed = pars[i]
        if parsed['category'] ==48 :
            Dict['date'] = k[1]
            if 'I040' in parsed.keys():
                Dict['RHO'] = parsed['I040']['RHO']['val']
                Dict['THETA'] = parsed['I040']['THETA']['val']
            else:
                Dict['RHO'] = np.nan
                Dict['THETA'] = np.nan
            Dict['TS']= k[2]
            Dict['len'] = parsed['len']
            Dict['crc'] = parsed['crc']
            if 'I090' in parsed.keys():
                Dict['FL'] = parsed['I090']['FL']['val']
            else :
                Dict['FL'] = np.nan
            Dict['SAC'] = parsed['I010']['SAC']['val']
            Dict['SIC'] = parsed['I010']['SIC']['val']
            Dict['TOD'] = parsed['I140']['ToD']['val']
            Dict['TYP'] = parsed['I020']['TYP']['val']
            if 'I240' in parsed.keys():
                Dict['TID'] = parsed['I240']['TId']['val']
            else :
                Dict['TID'] = np.nan
            if 'I200' in parsed.keys():
                Dict['CGS'] = parsed['I200']['CGS']['val']
                Dict['CHdg'] = parsed['I200']['CHdg']['val']
            else: 
                Dict['CGS'] = np.nan
                Dict['CHdg'] = np.nan
            Dataset.append(Dict)
        Dict={}

In [8]:
df = pd.DataFrame(Dataset)


In [9]:
len(df)

116003

In [10]:
df.shape

(116003, 14)

In [11]:
df['TID'].value_counts()

BCS28P      50705
AFR225       2665
TAY4216      2401
AFR257       2373
TAY4958      1917
            ...  
FPO4782         3
JMP232          3
EDYY            3
HUAF651         2
KLM702          2
Name: TID, Length: 237, dtype: int64

In [12]:
df.to_csv('data.csv')